In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'

In [4]:
# Load the CSV file
data = pd.read_csv(csv_file)

In [5]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.2, random_state=42)


In [6]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 2728 validated image filenames belonging to 62 classes.


In [7]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 682 validated image filenames belonging to 62 classes.


In [8]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(400, 300, 3))


29084464/29084464 [==============================] - 1s 0us/step


In [9]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Add custom layers with dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout with 50% rate
output = Dense(62, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [11]:
# Set a custom learning rate
learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)

In [12]:
# Compile the model with the custom learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [14]:
# Train the model with early stopping
history = model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping])

Epoch 1/50
86/86 [==============================] - 1782s 21s/step - loss: 4.4748 - accuracy: 0.0242 - val_loss: 4.0489 - val_accuracy: 0.0499
Epoch 2/50
86/86 [==============================] - 62s 718ms/step - loss: 4.1656 - accuracy: 0.0389 - val_loss: 3.8330 - val_accuracy: 0.0968
Epoch 3/50
86/86 [==============================] - 60s 704ms/step - loss: 3.9962 - accuracy: 0.0535 - val_loss: 3.6668 - val_accuracy: 0.1613
Epoch 4/50
86/86 [==============================] - 60s 700ms/step - loss: 3.8022 - accuracy: 0.0722 - val_loss: 3.5176 - val_accuracy: 0.2258
Epoch 5/50
86/86 [==============================] - 69s 808ms/step - loss: 3.6582 - accuracy: 0.1056 - val_loss: 3.3755 - val_accuracy: 0.3079
Epoch 6/50
86/86 [==============================] - 61s 705ms/step - loss: 3.5347 - accuracy: 0.1345 - val_loss: 3.2440 - val_accuracy: 0.3607
Epoch 7/50
86/86 [==============================] - 61s 704ms/step - loss: 3.3918 - accuracy: 0.1602 - val_loss: 3.1219 - val_accuracy: 0.4003